<a href="https://colab.research.google.com/github/shahafp/Conala-Challenge/blob/master/Conala_UnTokenized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import re
import torch
import numpy
from torch import autograd, nn, optim
import torch.nn.functional as F
import re
import random
import time
import math
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [0]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 100

In [0]:
class VocabIntent :
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


In [0]:
class VocabCode:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        # for word in re.split('([^a-zA-Z0-9 ])',sentence):
        #   if word is not '':
        self.addWord(sentence)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
def orginize_data(data_type):
  json_data = '/content/drive/My Drive/conala/' + data_type  
  path = open(json_data, "r")
  data = json.load(path)
  pairs=[]
  for dic in data:
      if dic["rewritten_intent"] is None:
          continue
      pairs.append([dic["rewritten_intent"], dic["snippet"]])
  orginize_data_mined(pairs)

def orginize_data_mined(pairs):
  json_data = 'C:\\Users\\SPariente\\Desktop\\Work\\Dev\\Python\\NLP\\Conala\\conala-corpus\\conala-mined.jsonl'
  with open(json_data, 'r', encoding='utf-8') as f:
      for line in f:
          j_line = json.loads(line.rstrip('\n|\r'))
          pairs.append([j_line["intent"], j_line["snippet"]])
  return pairs

In [0]:
def prepare_data(vocab1_name, vocab2_name, data_type):
  pairs=orginize_data(data_type=data_type)
  intent_vocab=VocabIntent(vocab1_name)
  code_vocab=VocabCode(vocab2_name)

  for pair in pairs:
    intent_vocab.addSentence(pair[0])
    code_vocab.addSentence(pair[1])
  
  return intent_vocab, code_vocab, pairs

In [0]:
input_lang, output_lang, pairs= prepare_data('intent', 'code', 'conala-train.json')

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def c_indexesFromSentence(lang, sentence):
    return [lang.word2index[sentence]]

def tensorFromSentence(lang, sentence, key):
    if key is 'intent':
      indexes = indexesFromSentence(lang, sentence)
      indexes.append(EOS_token)
    else:
      indexes = c_indexesFromSentence(lang, sentence)
      indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0], 'intent')
    target_tensor =tensorFromSentence(output_lang, pair[1], 'code')
    return (input_tensor, target_tensor)


In [0]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

      showPlot(plot_losses)

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence, 'intent')
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence.strip(' '))
        print('')

In [0]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=100)

0m 5s (- 73m 30s) (100 0%) 4.9756
0m 7s (- 44m 1s) (200 0%) 4.1142
0m 8s (- 34m 5s) (300 0%) 3.9351
0m 9s (- 29m 13s) (400 0%) 3.8974
0m 10s (- 26m 13s) (500 0%) 3.8946
0m 11s (- 24m 20s) (600 0%) 3.9126
0m 12s (- 22m 56s) (700 0%) 3.9114
0m 14s (- 21m 51s) (800 1%) 3.8960
0m 15s (- 21m 6s) (900 1%) 3.8851
0m 16s (- 20m 24s) (1000 1%) 3.8685
0m 17s (- 19m 55s) (1100 1%) 3.9122
0m 19s (- 19m 44s) (1200 1%) 3.8965
0m 20s (- 19m 18s) (1300 1%) 3.8879
0m 21s (- 18m 55s) (1400 1%) 3.9068
0m 22s (- 18m 39s) (1500 2%) 3.8795
0m 23s (- 18m 20s) (1600 2%) 3.8369
0m 25s (- 18m 3s) (1700 2%) 3.8541
0m 26s (- 17m 52s) (1800 2%) 3.8901
0m 27s (- 17m 37s) (1900 2%) 3.8872
0m 28s (- 17m 26s) (2000 2%) 3.8885
0m 30s (- 17m 25s) (2100 2%) 3.8540
0m 31s (- 17m 15s) (2200 2%) 3.8250
0m 32s (- 17m 12s) (2300 3%) 3.8436
0m 33s (- 17m 4s) (2400 3%) 3.8321
0m 35s (- 16m 57s) (2500 3%) 3.8710
0m 36s (- 16m 51s) (2600 3%) 3.8202
0m 37s (- 16m 43s) (2700 3%) 3.8530
0m 38s (- 16m 38s) (2800 3%) 3.8634
0m 39s (- 

In [0]:
evaluateRandomly(encoder1, attn_decoder1)

> sort a multidimensional array `a` by column with index 1
= sorted(a, key=lambda x: x[1])
< sorted(a, key=lambda x: x[1]) <EOS>

> getting every possible combination of two elements in a list
= list(itertools.combinations([1, 2, 3, 4, 5, 6], 2))
< list(itertools.combinations([1, 2, 3, 4, 5, 6], 2)) <EOS>

> Calling an external command "echo Hello World"
= return_code = subprocess.call('echo Hello World', shell=True)
< print(os.popen('echo Hello World').read()) <EOS>

> calculate the sum of the squares of each value in list `l`
= sum(map(lambda x: x * x, l))
< sum(map(lambda x: x * x, l)) <EOS>

> assign value in `group` dynamically to class property `attr`
= setattr(self, attr, group)
< setattr(self, attr, group) <EOS>

> prompt string 'Press Enter to continue...' to the console
= input('Press Enter to continue...')
< input('Press Enter to continue...') <EOS>

> check if a local variable `myVar` exists
= ('myVar' in locals())
< ('myVar' in globals()) <EOS>

> convert a DateTime string

In [0]:
evaluateRandomly(encoder1, attn_decoder1)

> Get all the second values from a list of lists `A`
= [row[1] for row in A]
< [row[1] for row in A] <EOS>

> print a list of integers `list_of_ints` using string formatting
= print(', '.join(str(x) for x in list_of_ints))
< print(', '.join(str(x) for x in list_of_ints)) <EOS>

> cut a string using delimiter '&'
= s[:s.rfind('&')]
< s[:s.rfind('&')] <EOS>

> return dataframe `df` with last row dropped
= df.ix[:-1]
< df.ix[:-1] <EOS>

> Create a dictionary from string `e` separated by `-` and `,`
= dict((k, int(v)) for k, v in (e.split(' - ') for e in s.split(',')))
< dict((k, int(v)) for k, v in (e.split(' - ') for e in s.split(','))) <EOS>

> send data 'HTTP/1.0 200 OK\r\n\r\n' to socket `connection`
= connection.send('HTTP/1.0 200 established\r\n\r\n')
< connection.send('HTTP/1.0 200 established\r\n\r\n') <EOS>

> Get the first and last 3 elements of list `l`
= l[:3] + l[-3:]
< l[:3] + l[-3:] <EOS>

> Print variable `count` and variable `conv` with space string '    ' in between
= pr